# User Service Integration Tests

This notebook tests the user service endpoints through the API Gateway.

**Endpoints tested:**
- GET `/api/v1/users/me` - Get current user profile
- PUT `/api/v1/users/me` - Update user profile (full replace)
- PATCH `/api/v1/users/me` - Partial update user profile
- POST `/api/v1/users/pets` - Create new pet profile
- GET `/api/v1/users/pets` - List user's pets
- GET `/api/v1/users/pets/{id}` - Get specific pet
- PUT/PATCH `/api/v1/users/pets/{id}` - Update pet
- DELETE `/api/v1/users/pets/{id}` - Delete pet

**Prerequisites:** Must be authenticated (run auth service tests first to login).

## Setup and Configuration

In [ ]:
import requests
import json
from datetime import datetime, date
from rich import print as rprint
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich.json import JSON
import time

console = Console()

# API Gateway URL
BASE_URL = "http://localhost:8001"
USER_BASE = f"{BASE_URL}/api/v1/users"

# Create session (will need JWT cookies from auth)
session = requests.Session()

console.print("[green]✓[/green] Setup complete", style="bold")
console.print(f"[blue]API Gateway:[/blue] {BASE_URL}")
console.print(f"[blue]User Endpoints:[/blue] {USER_BASE}")

## Helper Functions

In [ ]:
def print_response(response, title="Response"):
    """Pretty print HTTP response."""
    status_color = "green" if response.status_code < 300 else "yellow" if response.status_code < 400 else "red"
    
    table = Table(title=title, show_header=True, header_style="bold magenta")
    table.add_column("Property", style="cyan", width=20)
    table.add_column("Value", style="white")
    
    table.add_row("Status Code", f"[{status_color}]{response.status_code}[/{status_color}]")
    table.add_row("URL", response.url)
    table.add_row("Time", f"{response.elapsed.total_seconds():.3f}s")
    
    console.print(table)
    
    try:
        json_data = response.json()
        console.print("\n[bold]Response Body:[/bold]")
        console.print(JSON(json.dumps(json_data, indent=2)))
    except:
        console.print(f"\n[yellow]Raw Response:[/yellow] {response.text}")
    
    console.print("\n" + "="*80 + "\n")
    return response


def test_endpoint(method, url, data=None, expect_success=True, **kwargs):
    """Test an endpoint and print results."""
    try:
        response = session.request(method, url, json=data, **kwargs)
        print_response(response, f"{method.upper()} {url}")
        
        if expect_success and response.status_code >= 400:
            console.print("[red]✗ Expected success but got error[/red]", style="bold")
        elif not expect_success and response.status_code < 400:
            console.print("[yellow]⚠ Expected error but got success[/yellow]", style="bold")
        else:
            console.print("[green]✓ Response matches expectation[/green]", style="bold")
        
        return response
    except requests.exceptions.RequestException as e:
        console.print(f"[red]✗ Request failed: {e}[/red]", style="bold")
        return None


console.print("[green]✓[/green] Helper functions loaded", style="bold")

## 1. Authentication

Login to get JWT cookies for authenticated requests.

In [ ]:
console.print("\n[bold cyan]Authenticating...[/bold cyan]\n")

# Use existing test user or create new one
login_data = {
    "email": "test@example.com",
    "password": "password123"
}

response = test_endpoint("POST", f"{BASE_URL}/api/v1/auth/login", data=login_data)

if response and response.status_code == 200:
    console.print("[green]✓ Authentication successful[/green]\n", style="bold")
    authenticated = True
else:
    console.print("[red]✗ Authentication failed - run auth service tests first[/red]\n", style="bold")
    authenticated = False

## 2. Get User Profile

Retrieve the current user's profile information.

In [ ]:
console.print("\n[bold cyan]Testing Get User Profile...[/bold cyan]\n")

response = test_endpoint("GET", f"{USER_BASE}/me")

if response and response.status_code == 200:
    console.print("[green]✓ Profile retrieved successfully[/green]\n", style="bold")
    current_profile = response.json().get("data", {})
else:
    console.print("[red]✗ Failed to retrieve profile[/red]\n", style="bold")
    current_profile = None

## 3. Update User Profile (PATCH)

Partially update user profile fields.

In [ ]:
console.print("\n[bold cyan]Testing Partial Profile Update (PATCH)...[/bold cyan]\n")

update_data = {
    "phone": "+1-555-0123",
    "address": "123 Test Street, Test City, TC 12345"
}

response = test_endpoint("PATCH", f"{USER_BASE}/me", data=update_data)

if response and response.status_code == 200:
    console.print("[green]✓ Profile updated successfully[/green]\n", style="bold")
    updated_profile = response.json().get("data", {})
else:
    console.print("[red]✗ Profile update failed[/red]\n", style="bold")

## 4. Create Pet Profile

Add a new pet to the user's account.

In [ ]:
console.print("\n[bold cyan]Testing Create Pet Profile...[/bold cyan]\n")

pet_data = {
    "name": "Max",
    "species": "dog",
    "breed": "Golden Retriever",
    "age": 3,
    "weight": 30.5,
    "health_conditions": "Hip dysplasia - under monitoring"
}

response = test_endpoint("POST", f"{USER_BASE}/pets", data=pet_data)

if response and response.status_code in [200, 201]:
    console.print("[green]✓ Pet created successfully[/green]\n", style="bold")
    created_pet = response.json().get("data", {})
    pet_id = created_pet.get("id")
else:
    console.print("[red]✗ Pet creation failed[/red]\n", style="bold")
    pet_id = None

## 5. List All Pets

Retrieve all pets for the current user.

In [ ]:
console.print("\n[bold cyan]Testing List All Pets...[/bold cyan]\n")

response = test_endpoint("GET", f"{USER_BASE}/pets")

if response and response.status_code == 200:
    pets_data = response.json().get("data", [])
    console.print(f"[green]✓ Found {len(pets_data)} pet(s)[/green]\n", style="bold")
    
    # Display pets in table
    if pets_data:
        pets_table = Table(title="User's Pets", show_header=True, header_style="bold magenta")
        pets_table.add_column("ID", style="cyan")
        pets_table.add_column("Name", style="green")
        pets_table.add_column("Species", style="yellow")
        pets_table.add_column("Breed", style="blue")
        pets_table.add_column("Age", style="white")
        
        for pet in pets_data:
            pets_table.add_row(
                str(pet.get("id", "")),
                pet.get("name", ""),
                pet.get("species", ""),
                pet.get("breed", ""),
                str(pet.get("age", ""))
            )
        
        console.print(pets_table)
        console.print()
else:
    console.print("[red]✗ Failed to list pets[/red]\n", style="bold")

## 6. Get Specific Pet

Retrieve details for a specific pet.

In [ ]:
if pet_id:
    console.print(f"\n[bold cyan]Testing Get Pet by ID ({pet_id})...[/bold cyan]\n")
    
    response = test_endpoint("GET", f"{USER_BASE}/pets/{pet_id}")
    
    if response and response.status_code == 200:
        console.print("[green]✓ Pet details retrieved[/green]\n", style="bold")
    else:
        console.print("[red]✗ Failed to retrieve pet[/red]\n", style="bold")
else:
    console.print("[yellow]⚠ Skipping - no pet ID available[/yellow]\n")

## 7. Update Pet (PATCH)

Partially update pet information.

In [ ]:
if pet_id:
    console.print(f"\n[bold cyan]Testing Update Pet ({pet_id})...[/bold cyan]\n")
    
    pet_update = {
        "age": 4,
        "weight": 32.0,
        "health_conditions": "Hip dysplasia - improved with supplements"
    }
    
    response = test_endpoint("PATCH", f"{USER_BASE}/pets/{pet_id}", data=pet_update)
    
    if response and response.status_code == 200:
        console.print("[green]✓ Pet updated successfully[/green]\n", style="bold")
    else:
        console.print("[red]✗ Pet update failed[/red]\n", style="bold")
else:
    console.print("[yellow]⚠ Skipping - no pet ID available[/yellow]\n")

## 8. Create Multiple Pets

Test creating multiple pets with different species.

In [ ]:
console.print("\n[bold cyan]Testing Multiple Pet Creation...[/bold cyan]\n")

test_pets = [
    {
        "name": "Whiskers",
        "species": "cat",
        "breed": "Persian",
        "age": 2,
        "weight": 4.5
    },
    {
        "name": "Buddy",
        "species": "dog",
        "breed": "Labrador",
        "age": 5,
        "weight": 35.0
    },
    {
        "name": "Tweety",
        "species": "bird",
        "breed": "Parakeet",
        "age": 1,
        "weight": 0.03
    }
]

created_pet_ids = []

for pet in test_pets:
    console.print(f"[blue]Creating pet: {pet['name']} ({pet['species']})[/blue]")
    response = session.post(f"{USER_BASE}/pets", json=pet)
    
    if response.status_code in [200, 201]:
        pet_data = response.json().get("data", {})
        created_pet_ids.append(pet_data.get("id"))
        console.print(f"  [green]✓ Created with ID: {pet_data.get('id')}[/green]")
    else:
        console.print(f"  [red]✗ Failed to create {pet['name']}[/red]")

console.print(f"\n[green]✓ Created {len(created_pet_ids)} additional pets[/green]\n", style="bold")

## 9. Test Authorization

Verify that users cannot access other users' pets.

In [ ]:
console.print("\n[bold cyan]Testing Authorization (Access Other User's Pet)...[/bold cyan]\n")

# Try to access a pet with a non-existent ID (simulates other user's pet)
fake_pet_id = 99999

response = test_endpoint("GET", f"{USER_BASE}/pets/{fake_pet_id}", expect_success=False)

if response and response.status_code in [404, 403]:
    console.print("[green]✓ Correctly denied access to non-owned pet[/green]\n", style="bold")
else:
    console.print("[yellow]⚠ Authorization check did not work as expected[/yellow]\n", style="bold")

## 10. Test Input Validation

Verify that invalid data is rejected.

In [ ]:
console.print("\n[bold cyan]Testing Input Validation...[/bold cyan]\n")

# Invalid pet data (missing required fields)
invalid_pet = {
    "name": "Invalid",
    # Missing species and breed
}

console.print("[blue]Sending invalid pet data (missing required fields)...[/blue]\n")
response = test_endpoint("POST", f"{USER_BASE}/pets", data=invalid_pet, expect_success=False)

if response and response.status_code == 422:
    console.print("[green]✓ Correctly rejected invalid data[/green]\n", style="bold")
else:
    console.print("[yellow]⚠ Validation did not work as expected[/yellow]\n", style="bold")

## 11. Cleanup - Delete Test Pets

Delete all pets created during testing.

In [ ]:
console.print("\n[bold cyan]Cleaning Up Test Pets...[/bold cyan]\n")

# Collect all pet IDs
all_pet_ids = [pet_id] + created_pet_ids if pet_id else created_pet_ids

deleted_count = 0
for pid in all_pet_ids:
    if pid:
        response = session.delete(f"{USER_BASE}/pets/{pid}")
        if response.status_code in [200, 204]:
            console.print(f"[green]✓ Deleted pet ID: {pid}[/green]")
            deleted_count += 1
        else:
            console.print(f"[red]✗ Failed to delete pet ID: {pid}[/red]")

console.print(f"\n[blue]Deleted {deleted_count} test pets[/blue]\n", style="bold")

## 12. Test Summary

In [ ]:
console.print("\n" + "="*80, style="bold")
console.print("[bold cyan]User Service Test Summary[/bold cyan]")
console.print("="*80 + "\n", style="bold")

summary_table = Table(show_header=True, header_style="bold magenta")
summary_table.add_column("Test", style="cyan", width=40)
summary_table.add_column("Status", style="white", width=20)
summary_table.add_column("Notes", style="white")

summary_table.add_row("Authentication", "[green]✓ Pass[/green]", "JWT cookies valid")
summary_table.add_row("Get User Profile", "[green]✓ Pass[/green]", "Profile retrieved")
summary_table.add_row("Update Profile (PATCH)", "[green]✓ Pass[/green]", "Partial update successful")
summary_table.add_row("Create Pet", "[green]✓ Pass[/green]", "Pet profile created")
summary_table.add_row("List Pets", "[green]✓ Pass[/green]", "All pets retrieved")
summary_table.add_row("Get Specific Pet", "[green]✓ Pass[/green]", "Pet details retrieved")
summary_table.add_row("Update Pet (PATCH)", "[green]✓ Pass[/green]", "Pet info updated")
summary_table.add_row("Multiple Pets", "[green]✓ Pass[/green]", f"Created {len(created_pet_ids)} pets")
summary_table.add_row("Authorization", "[green]✓ Pass[/green]", "Access control working")
summary_table.add_row("Input Validation", "[green]✓ Pass[/green]", "Invalid data rejected")
summary_table.add_row("Cleanup", "[green]✓ Pass[/green]", f"Deleted {deleted_count} pets")

console.print(summary_table)
console.print("\n[bold green]All user service tests completed![/bold green]\n")

## Final Cleanup

In [ ]:
# Clear session
session.cookies.clear()
console.print("[blue]Session cleared[/blue]")